In [7]:
!wget http://www.kasprowski.pl/datasets/emvic.zip
!unzip emvic.zip


--2024-10-19 16:22:54--  http://www.kasprowski.pl/datasets/emvic.zip
Resolving www.kasprowski.pl (www.kasprowski.pl)... 94.152.164.213
Connecting to www.kasprowski.pl (www.kasprowski.pl)|94.152.164.213|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4295994 (4.1M) [application/zip]
Saving to: ‘emvic.zip.1’

emvic.zip.1         100%[===================>]   4.10M  3.51MB/s    in 1.2s    

2024-10-19 16:22:55 (3.51 MB/s) - ‘emvic.zip.1’ saved [4295994/4295994]

Archive:  emvic.zip
replace emvic.data? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: emvic.data              


In [20]:
# import pandas as pd
# from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from sklearn.feature_selection import SelectKBest, f_classif
# from sklearn.metrics import accuracy_score
# from sklearn.naive_bayes import GaussianNB
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.neighbors import KNeighborsClassifier
# import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import numpy as np

In [21]:
# Wczytanie danych
data = pd.read_csv('emvic.data', sep='\t', header=None)

print(data.head())


  0      1      2      3      4      5      6      7      8      9      ...  \
0   a25      0      0      0      0      0      0      0      0      0  ...   
1   a25      0      0      0      0      0      0      0      0      0  ...   
2   a25      0      0      0      0      0      0      0      0      0  ...   
3   a25      0      0      0      0      0      0      0      0      0  ...   
4   a25      0      0      0      0      0      0      0      0      0  ...   

   12279  12280  12281  12282  12283  12284  12285  12286  12287  12288  
0    -20    -20    -20    -20    -19    -20    -22    -19    -22    -23  
1    -74    -73    -75    -73    -72    -72    -69    -67    -67    -70  
2    -51    -52    -54    -51    -54    -54    -54    -53    -52    -54  
3     74     74     74     74     74     74     75     71     76     74  
4    129    130    130    133    131    129    124    124    122    117  

[5 rows x 12289 columns]


In [22]:

X = data.iloc[:, 1:]
y = data.iloc[:, 0]

X['difference_lr'] = X.iloc[:, 2] - X.iloc[:, 3]  # Różnica między lewym a prawym okiem
X['speed'] = X.diff(axis=1).mean(axis=1)  # Przybliżona prędkość ruchu oka
X['acceleration'] = X.diff(axis=1).diff(axis=1).mean(axis=1)  # Przybliżone przyspieszenie ruchu oka

# Usunięcie kolumn, które mają stałe wartości
X = X.loc[:, (X != X.iloc[0]).any()]

#skalowanie:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# podział na testowy i treningowy:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42)


print(X_train)
print('\n')
print(X_scaled)



[[-0.13082398 -0.21876687 -0.37796447 ...  0.1366403   0.11468786
   0.12654185]
 [-0.13082398 -0.21876687 -0.37796447 ... -0.12548409 -0.12453656
  -0.12615889]
 [-0.13082398 -0.21876687 -0.37796447 ...  0.38220947  0.38140934
   0.35452188]
 ...
 [-0.13082398 -0.21876687 -0.37796447 ... -3.7234863  -3.74039996
  -3.74362503]
 [-0.13082398 -0.21876687  2.64575131 ... -1.27883143 -1.26016515
  -1.24683177]
 [-0.13082398 -0.21876687 -0.37796447 ... -0.55591994 -0.5534907
  -0.55465147]]


[[-0.13082398 -0.21876687 -0.37796447 ... -0.24688908 -0.25377275
  -0.25525601]
 [-0.13082398 -0.21876687 -0.37796447 ... -0.37933088 -0.37750952
  -0.38435314]
 [-0.13082398 -0.21876687 -0.37796447 ... -0.34070202 -0.33626393
  -0.34040518]
 ...
 [-0.13082398 -0.21876687 -0.37796447 ...  0.43739355  0.43090405
   0.42868405]
 [-0.13082398 -0.21876687 -0.37796447 ...  1.03614085  1.03033996
   1.0302217 ]
 [-0.13082398 -0.21876687 -0.37796447 ...  1.39207818  1.39330115
   1.39553909]]


In [ ]:
# feature selection - RFE:

rf = RandomForestClassifier(random_state=42)
selector = RFE(rf, n_features_to_select=50)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)


In [18]:

#Wykorzystanie klasyfikatora Random Forest:
rf_param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

rf_random_search = RandomizedSearchCV(RandomForestClassifier(random_state=42), rf_param_dist, n_iter=50, cv=5, random_state=42)
rf_random_search.fit(X_train_selected, y_train)
y_pred_rf = rf_random_search.predict(X_test_selected)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Dokładność Random Forest (optymalizowane): {accuracy_rf:.4f}")

Dokładność Random Forest (optymalizowane): 0.4808


In [ ]:
#Wykorzystanie klasyfikatora SVC:
svc_param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.01, 0.1, 1],
    'kernel': ['rbf', 'linear']
}
svc_grid_search = GridSearchCV(SVC(random_state=42), svc_param_grid, cv=5)
svc_grid_search.fit(X_train_selected, y_train)
y_pred_svc = svc_grid_search.predict(X_test_selected)
accuracy_svc = accuracy_score(y_test, y_pred_svc)
print(f"Accuracy of SVC (optimized): {accuracy_svc:.4f}")


Accuracy of SVC (optimized): 0.3462


In [ ]:
#Wykorzystanie klasyfikatora Logistic Regression:
lr_clf = LogisticRegression(max_iter=10000)
lr_clf.fit(X_train_selected, y_train)
y_pred_lr = lr_clf.predict(X_test_selected)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f"Accuracy of Logistic Regression: {accuracy_lr:.4f}")

Accuracy of Logistic Regression: 0.3942


In [ ]:
#Wykorzystanie klasyfikatora Naive Bayes
nb_clf = GaussianNB()
nb_clf.fit(X_train_selected, y_train)
y_pred_nb = nb_clf.predict(X_test_selected)
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print(f"Accuracy of Naive Bayes: {accuracy_nb:.4f}")

Accuracy of Naive Bayes: 0.2692


In [ ]:
# Wykorzystanie klasyfikatora  MLP:
mlp_clf = MLPClassifier(max_iter=1000, random_state=42)
mlp_clf.fit(X_train_selected, y_train)
y_pred_mlp = mlp_clf.predict(X_test_selected)
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
print(f"Accuracy of MLP Classifier: {accuracy_mlp:.4f}")

Accuracy of MLP Classifier: 0.3942


In [ ]:
#Wykorzystanie klasyfikatora Decision Tree
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train_selected, y_train)
y_pred_dt = dt_clf.predict(X_test_selected)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print(f"Accuracy of Decision Tree: {accuracy_dt:.4f}")

Accuracy of Decision Tree: 0.2596


In [ ]:
#Wykorzystanie klasyfikatora KNN:
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train_selected, y_train)
y_pred_knn = knn_clf.predict(X_test_selected)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f"Accuracy of KNeighbors: {accuracy_knn:.4f}")

Accuracy of KNeighbors: 0.3173


In [ ]:
# cross walidacja:
for clf, name in [(rf_clf, 'Random Forest'), (svc_clf, 'SVC')]:
    scores = cross_val_score(clf, X_train_selected, y_train, cv=5)
    print(f"Cross-validation accuracy of {name}: {np.mean(scores):.4f}")

Cross-validation accuracy of Random Forest: 0.3781
Cross-validation accuracy of SVC: 0.3589
